In [ ]:
import gc
import os

import dask
import dask.array as da
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster, progress

gc.collect()

os.chdir(
    "/mnt/cloud/wwu1/ec_bronze/_nogroup/ae78a1ca-a0e8-4e4e-8992-69c34947db65/UseCase_AIRCON"
)

# relative humidity
def rh(t, d):
    return (da.exp((17.625 * d) / (243.04 + t)) / (da.exp((17.625 * t) / (243.04 + t)))) * 100


scale_factor_rh = 100 / (2**16 - 1)  # applies to all datasets

# Define your open_mfdataset parameters
chunks = {"longitude": "auto", "latitude": "auto", "time": "auto"}

In [ ]:
z = rh(22, 20)
z

In [ ]:
client = Client()
print(client)

In [ ]:
y = 2022
# Open the datasets using xarray.open_mfdataset() and Dask
d = xr.open_mfdataset(
    [
        f"supplementary/era5/ERA5_2_metre_temperature_{y}.nc",
        f"supplementary/era5/ERA5_2_metre_dewpoint_temperature_{y}.nc",
    ],
    chunks=chunks,
    combine="by_coords",
    parallel=True,
)  # .isel(time=range(0,64))

d

In [ ]:
%%time

d["rh"] = rh(d.t2m, d.d2m)

In [ ]:
out_rh = f"supplementary/rel_hum/ERA5_2_metre_Rel_Humidity_{y}.nc"
rh = (
    d["rh"]
    .to_dataset()
    .to_netcdf(
        out_rh,
        compute=False,
        encoding={
            "rh": {
                "dtype": "int16",
                "scale_factor": scale_factor_rh,
                "_FillValue": -9999,
            }
        },
    )
)

In [ ]:
# ws_res = client.gather(client.persist(ws))
rh_res = rh.persist()
progress(rh_res)

In [ ]:
d.close()

In [ ]:
r = xr.open_dataset(out_rh)
r

In [ ]:
# client.close()